In [24]:
import pandas as pd

# Source: https://latitudelongitude.org/ch/zuerich-kreis-12/

# ID : lon, lat
dict_coords = {
    1 : [47.37055, 8.54177],
    2 : [47.33756, 8.5211],
    3 : [47.35785, 8.50296],
    4 : [47.37752, 8.5212],
    5 : [47.38767, 8.52152],
    6 : [47.39223, 8.54381],
    7 : [47.37328, 8.58038],
    8 : [47.3548, 8.56097],
    9 : [47.38245, 8.47993],
    10 : [47.40773, 8.5005],
    11 : [47.42326, 8.52166],
    12 : [47.40372, 8.57608]
}

latid, longi, ids = [], [], []
for key, values in dict_coords.items():
    ids.append(key)
    latid.append(values[0])
    longi.append(values[1])


df = pd.DataFrame()
df['id'] = ids
df['lat'] = latid
df['lon'] = longi
df['color'] = [1 for i in range(12)]
df
None

In [23]:
import json
import plotly.graph_objects as go


with open('stadtkreise.json') as json_file:
    stadtkreise = json.load(json_file)

def set_ID(kreis):
    for i in range(len(kreis['features'])):
        kreis['features'][i]['id'] = kreis['features'][i]['properties']['name']
    return kreis

stadtkreise = set_ID(stadtkreise)

fig = go.Figure(go.Choroplethmapbox(geojson=stadtkreise, locations=df.id, z=df.color,
                                    colorscale="Viridis",
                                    marker_opacity=0.5, marker_line_width=0))


fig.add_scattermapbox(lat = df.lat.tolist(),
                      lon = df.lon.tolist(),
                      line_color = 'red',
                      mode="markers+text",
                      marker={"size": 7})



fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=10, mapbox_center = {"lat": 47.4, "lon": 8.6})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
None